<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/section_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sections as Input

In [2]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [3]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 2.6 kB/s 


In [4]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Download Data

### Basic Chords (major triads, minor triads, dom7)

In [7]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO' -O data.csv

--2021-11-18 19:43:21--  https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO
Resolving docs.google.com (docs.google.com)... 209.85.147.100, 209.85.147.102, 209.85.147.138, ...
Connecting to docs.google.com (docs.google.com)|209.85.147.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/50coihaak5a5bm6kqjiqukcpfnc56dql/1637264550000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download [following]
--2021-11-18 19:43:22--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/50coihaak5a5bm6kqjiqukcpfnc56dql/1637264550000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c2f::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-

### Simplified Chords

In [8]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb' -O data.csv

--2021-11-18 19:43:22--  https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb
Resolving docs.google.com (docs.google.com)... 74.125.124.101, 74.125.124.138, 74.125.124.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.124.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k0v17m5g7p6u8bpd194ovmg5khtv6obb/1637264550000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download [following]
--2021-11-18 19:43:23--  https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k0v17m5g7p6u8bpd194ovmg5khtv6obb/1637264550000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download
Resolving doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-docs.googleusercontent.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-

In [9]:
path_to_file = '/content/data.csv'
df = pd.read_csv(path_to_file, sep='\t', index_col="id")
df = df.reset_index()

In [10]:
df

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 G#7 C#M7 E7 ...
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm7 C7 Bm7 E7 AM7 C7 FM7 Bbm7 Eb7 G#M7 Dm7 G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am7 Am7 Cm7 Cm7 EbM7 EbM7 Em7b5 A7 Dm7 Dm7 Bm7...
...,...,...,...,...,...,...,...,...
5398,5398,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,1,Dm7 G7 Em7b5 A7 Dm7 G#7 G7 C C
5399,5399,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,2,Dm7 G7 Em7b5 A7 Dm7 G#7 G7 C C A7 A7 A7 A7 Dm7...
5400,5400,dataset/trad/You're Nobody Till Somebody Loves...,You're Nobody Till Somebody Loves You,major,1809,A,1,C E7 A7 A7 Dm G7 C Em7 C G7 Em7 Cdim Dm7 Dm7 D...
5401,5401,dataset/trad/Your Feet's Too Big.xml,Your Feet's Too Big,major,1810,NaN,0,G7 G7 C C G7 G7 C F G# C C G7 G7 C F C C G7 C ...


### Meta Data

In [11]:
titles = df.loc[:, ['id', 'tune_id', 'section_id', 'section_name', 'title', 'tune_mode']]

In [12]:
titles

,id,tune_id,section_id,section_name,title,tune_mode
0,0,0,1,A,26-2,major
1,1,0,2,A,26-2,major
2,2,0,3,B,26-2,major
3,3,0,4,A,26-2,major
4,4,1,0,NaN,500 Miles High,minor
...,...,...,...,...,...,...
5398,5398,1808,1,A,You're Lucky To Me,major
5399,5399,1808,2,A,You're Lucky To Me,major
5400,5400,1809,1,A,You're Nobody Till Somebody Loves You,major
5401,5401,1810,0,NaN,Your Feet's Too Big,major


In [13]:
##TODO Now a tune contains multiple rows

In [14]:
titles_dict = titles.to_dict()
sectionid_to_title = titles_dict['title']
sectionid_to_title[1170]

'Emily'

In [15]:
sectionid_to_title_id = titles_dict['tune_id']
sectionid_to_title_id[5396]

1807

In [16]:
titles_rows = titles.to_dict(orient='records')
titles_rows[0]

sectionid_to_section = []
for i, row in enumerate(titles_rows):
  name = f"{row['title']}, section{row['section_id']} ({row['section_name']})"
  sectionid_to_section.append(name)

sectionid_to_section[3]

'26-2, section4 (A)'

In [17]:
title_to_sectionid = {}

for row in titles.iterrows():
  title = row[1]['title']
  if title not in title_to_sectionid:
    title_to_sectionid[title] = [row[1]['id']]
  else:
    title_to_sectionid[title].append(row[1]['id'])


In [18]:
title_to_sectionid

{'26-2': [0, 1, 2, 3],
 '500 Miles High': [4],
 '502 Blues': [5],
 '52nd Street Theme': [6, 7, 8, 9],
 '9.20 Special': [10, 11, 12, 13],
 'A Ballad': [14, 15, 16, 17],
 'A Beautiful Friendship': [18, 19, 20, 21],
 'A Blossom Fell': [22, 23],
 'A Certain Smile': [24, 25, 26, 27],
 'A Child Is Born': [28, 29],
 'A Felicidade': [30, 31, 32, 33, 34],
 'A Fine Romance': [35, 36, 37, 38],
 'A Flower Is A Lovesome Thing': [39, 40, 41, 42],
 'A Foggy Day': [43, 44, 45, 46],
 'A Ghost Of A Chance': [47, 48, 49, 50],
 'A Kiss To Build A Dream On': [51, 52, 53, 54],
 'A Little Tear': [55, 56],
 'A Lovely Way To Spend An Evening': [57, 58, 59, 60],
 'A Night In Tunisia': [61, 62, 63, 64],
 'A Nightingale Sang In Berkeley Square': [65, 66, 67, 68],
 'A Pretty Girl Is Like A Melody': [69, 70],
 "A Sleepin' Bee": [71, 72, 73, 74],
 'A Smooth One': [75, 76, 77, 78],
 'A Sound For Sore Ears': [79, 80, 81],
 'A Sunday Kind Of Love': [82, 83, 84, 85],
 'A Taste Of Honey': [86, 87],
 'A Time For Love': [8

# List Tunes for Test Data

In [19]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_sectionid[reference], title_to_sectionid[compare]])
  

In [20]:
contrafacts_test[:5]

[[[0, 1, 2, 3], [810, 811, 812, 813]],
 [[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[104, 105, 106, 107], [209, 210, 211, 212]],
 [[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]]]

## Helpers for N-Grams

In [21]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [22]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

# TF-IDF

## Build Model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
lines = df.loc[:, 'chords'].tolist()
data = [line.split(' ') for line in lines]

In [25]:
for i in data[:10]:
  print(i)

['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7']
['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7']
['Gm7', 'C7', 'Bm7', 'E7', 'AM7', 'C7', 'FM7', 'Bbm7', 'Eb7', 'G#M7', 'Dm7', 'G7']
['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7']
['Am7', 'Am7', 'Cm7', 'Cm7', 'EbM7', 'EbM7', 'Em7b5', 'A7', 'Dm7', 'Dm7', 'Bm7b5', 'Bm7b5', 'Bbm7', 'Bbm7', 'Fm7', 'Fm7', 'E7', 'E7', 'Fm7', 'Fm7', 'C#M7', 'C#M7', 'Fm7', 'Fm7', 'C#M7', 'C#M7']
['Am7', 'C#M7', 'Bm7b5', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Cm7', 'F7', 'BbM7', 'G#m7', 'C#7', 'F#m7b5', 'B7', 'EM7', 'EM7', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Cm7', 'F7', 'BbM7', 'G#m7', 'C#7', 'F#m7b5', 'B7', 'Em7', 'Em7']
['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7', 'C']
['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', '

In [26]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
tfidf_vectorizer  = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  print(data_ngrams[0])
  data_tfidf = [" ".join(i) for i in data_ngrams]

  # make sure that no processing on the chord symbols is done!
  tv = TfidfVectorizer(min_df = 0., 
                     max_df = 1., 
                     norm = 'l2', 
                     use_idf = True,
                     stop_words=None,
                     lowercase=False,
                     token_pattern='\\b[\\w#]+')

  tfidf_vectorizer[n] = tv
  tfidf_model[n] = tv.fit_transform(data_tfidf)

['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7']
['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7']
['CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7']


In [51]:
for i in range(len(tfidf_vectorizer)):
  print(tfidf_vectorizer[i+1].get_feature_names_out())

['A' 'A13' 'A6' 'A7' 'AM7' 'Adim' 'Adim7' 'Am' 'Am6' 'Am7' 'Am7b5' 'AmM7'
 'B' 'B13' 'B6' 'B7' 'BM7' 'Bb' 'Bb13' 'Bb6' 'Bb7' 'BbM7' 'Bbdim' 'Bbdim7'
 'Bbm' 'Bbm6' 'Bbm7' 'Bbm7b5' 'BbmM7' 'Bdim' 'Bdim7' 'Bm' 'Bm6' 'Bm7'
 'Bm7b5' 'BmM7' 'C' 'C#' 'C#13' 'C#6' 'C#7' 'C#M7' 'C#dim' 'C#dim7' 'C#m'
 'C#m6' 'C#m7' 'C#m7b5' 'C#mM7' 'C13' 'C6' 'C7' 'CM7' 'Cdim' 'Cdim7' 'Cm'
 'Cm6' 'Cm7' 'Cm7b5' 'CmM7' 'D' 'D13' 'D6' 'D7' 'DM7' 'Ddim' 'Ddim7' 'Dm'
 'Dm6' 'Dm7' 'Dm7b5' 'DmM7' 'E' 'E13' 'E6' 'E7' 'EM7' 'Eb' 'Eb13' 'Eb6'
 'Eb7' 'EbM7' 'Ebdim' 'Ebdim7' 'Ebm' 'Ebm6' 'Ebm7' 'Ebm7b5' 'EbmM7' 'Edim'
 'Edim7' 'Em' 'Em6' 'Em7' 'Em7b5' 'EmM7' 'F' 'F#' 'F#13' 'F#6' 'F#7'
 'F#M7' 'F#dim' 'F#dim7' 'F#m' 'F#m6' 'F#m7' 'F#m7b5' 'F#mM7' 'F13' 'F6'
 'F7' 'FM7' 'Fdim' 'Fdim7' 'Fm' 'Fm6' 'Fm7' 'Fm7b5' 'FmM7' 'G' 'G#' 'G#13'
 'G#6' 'G#7' 'G#M7' 'G#dim' 'G#dim7' 'G#m' 'G#m7' 'G#m7b5' 'G13' 'G6' 'G7'
 'GM7' 'Gdim' 'Gdim7' 'Gm' 'Gm6' 'Gm7' 'Gm7b5' 'GmM7' 'b5']
['A' 'A13' 'A6' 'A7' 'AM7' 'Adim' 'Adim7' 'Am' 'Am6' 'Am7' '

In [28]:
i = 1

tv_matrix = tfidf_model[i].toarray()

vocab = tfidf_vectorizer[i].get_feature_names_out()
tfidf_chords = pd.DataFrame(tv_matrix, columns = vocab)
tfidf_chords

,A,A13,A6,A7,AM7,Adim,Adim7,Am,Am6,Am7,Am7b5,AmM7,B,B13,B6,B7,BM7,Bb,Bb13,Bb6,Bb7,BbM7,Bbdim,Bbdim7,Bbm,Bbm6,Bbm7,Bbm7b5,BbmM7,Bdim,Bdim7,Bm,Bm6,Bm7,Bm7b5,BmM7,C,C#,C#13,C#6,...,F#dim7,F#m,F#m6,F#m7,F#m7b5,F#mM7,F13,F6,F7,FM7,Fdim,Fdim7,Fm,Fm6,Fm7,Fm7b5,FmM7,G,G#,G#13,G#6,G#7,G#M7,G#dim,G#dim7,G#m,G#m7,G#m7b5,G13,G6,G7,GM7,Gdim,Gdim7,Gm,Gm6,Gm7,Gm7b5,GmM7,b5
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.183958,0.0,0.0,0.0,0.0,0.0,0.244503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.219451,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.270663,0.355014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218613,0.0,0.0,0.0,0.0,0.0,0.247727,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.339563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.152386,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.493039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151803,0.0,0.0,0.0,0.0,0.0,0.172020,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.425097,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388876,0.0,0.0,0.0,0.0,0.0,0.0,0.354405,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.222138,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.359360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.110645,0.0,0.0,0.0,0.0,0.0,0.250760,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.339563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.152386,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.493039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151803,0.0,0.0,0.0,0.0,0.0,0.172020,0.0,0.0,0.0
4,0.0,0.0,0.0,0.054763,0.000000,0.0,0.0,0.000000,0.0,0.132782,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277299,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.199106,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.611458,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,0.0,0.0,0.0,0.215152,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.612088,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.383773,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309971,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5399,0.0,0.0,0.0,0.513191,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.486661,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.305132,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.369680,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5400,0.0,0.0,0.0,0.316703,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.525578,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190115,0.0,0.0,0.0,0.0

The `tfidf_model` is a list of the 4 models fitted with ngram size 1...4. 
Each of the model contains a sparse matrix with tunes being in the rows and the vocabulary being in the columns.

In [29]:
tfidf_model[1][5000]

<1x143 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [30]:
tfidf_model

{1: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43611 stored elements in Compressed Sparse Row format>,
 2: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43611 stored elements in Compressed Sparse Row format>,
 3: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43606 stored elements in Compressed Sparse Row format>,
 4: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43603 stored elements in Compressed Sparse Row format>}

## Test for single tunes

In [31]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

Evaluate TF-IDF Model for various ngram sizes.

In [32]:
model1 = 1
model2 = 2
model3 = 3
model4 = 4

df_sim = pd.DataFrame(columns=['reference_title',
                               'similar_title',
                               'section1', 
                               'section2', 
                               f'sim{model1}', 
                               f'sim{model2}',
                               f'sim{model3}',
                               f'sim{model4}',
                               ])

for tune in tunes_eval_list:
  print(tune)
  for s1 in title_to_sectionid[tune]:
      print(s1)
      for s2 in range(len(data_tfidf)):
        if s2 not in title_to_sectionid[tune]:
          sim1 = cosine_similarity(tfidf_model[model1][s1], tfidf_model[model1][s2])[0][0]
          sim2 = cosine_similarity(tfidf_model[model2][s1], tfidf_model[model2][s2])[0][0]
          sim3 = cosine_similarity(tfidf_model[model3][s1], tfidf_model[model3][s2])[0][0]
          sim4 = cosine_similarity(tfidf_model[model4][s1], tfidf_model[model4][s2])[0][0]
          if sim1 > 0.8:
            df_sim.loc[len(df_sim)] = [tune,
                                      sectionid_to_title[s2],
                                      sectionid_to_section[s1], 
                                      sectionid_to_section[s2], 
                                      sim1, 
                                      sim2,
                                      sim3,
                                      sim4,
                                      ]

df_sim = df_sim.sort_values(f'sim{model2}', ascending=False)

Sweet Sue, Just You
3752
3753
3754
3755
These Foolish Things
3866
3867
3868
3869
Blue Moon
496
497
498
499
All Of Me
198
199
200
201
All God's Chillun Got Rhythm
188
189
190
191
I Got Rhythm
1647
1648
1649
1650
4821
4822
4823
4824
Bye Bye Blackbird
653
654
4569
4570
4571
Old Fashioned Love
5056
Dinah
997
998
999
1000
Honeysuckle Rose
1538
1539
1540
1541
4793
4794
4795
4796
Misty
2676
2677
2678
2679


In [33]:
from google.colab import files

file = 'tfidf_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
df_sim.head(50)

,reference_title,similar_title,section1,section2,sim1,sim2,sim3,sim4
926,I Got Rhythm,Perdido,"I Got Rhythm, section3 (B)","Perdido, section3 (B)",1.000000,1.000000,1.000000,1.000000
927,I Got Rhythm,Rhythm-a-ning,"I Got Rhythm, section3 (B)","Rhythm-a-ning, section3 (B)",1.000000,1.000000,1.000000,1.000000
806,I Got Rhythm,I Love Being Here With You,"I Got Rhythm, section3 (B)","I Love Being Here With You, section3 (B)",1.000000,1.000000,1.000000,1.000000
936,I Got Rhythm,Two Not One,"I Got Rhythm, section3 (B)","Two Not One, section3 (B)",1.000000,1.000000,1.000000,1.000000
937,I Got Rhythm,Ain't That A Grand And Glorious Feeling,"I Got Rhythm, section3 (B)","Ain't That A Grand And Glorious Feeling, secti...",1.000000,1.000000,1.000000,1.000000
795,I Got Rhythm,Anthropology,"I Got Rhythm, section3 (B)","Anthropology, section3 (B)",1.000000,1.000000,1.000000,1.000000
802,I Got Rhythm,Flintstones,"I Got Rhythm, section3 (B)","Flintstones, section3 (B)",1.000000,1.000000,1.000000,1.000000
800,I Got Rhythm,Dexterity,"I Got Rhythm, section3 (B)","Dexterity, section3 (B)",1.000000,1.000000,1.000000,1.000000
941,I Got Rhythm,Sunday,"I Got Rhythm, section3 (B)","Sunday, section3 (B)",1.000000,1.000000,1.000000,1.000000
943,I Got Rhythm,When I'm Cleaning Windows,"I Got Rhythm, section3 (B)","When I'm Cleaning Windows, section2 (B)",1.000000,1.000000,1.000000,1.000000


## Test of the Contrafacts tunes

In [35]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)


In [36]:
for tune1, tune2 in contrafacts_test:
  print(f"{tune1}, {tune2}")

[0, 1, 2, 3], [810, 811, 812, 813]
[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[104, 105, 106, 107], [209, 210, 211, 212]
[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]
[594], [3746, 3747]
[668, 669, 670, 671], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[707, 708, 709, 710, 711, 712, 713, 714], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[835, 836, 837, 838], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[841], [4028]
[980, 981, 982, 983], [2968, 2969, 2970, 2971]
[984, 985, 986, 987], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[995, 996], [3746, 3747]
[1084, 1085], [1996, 1997]
[1042, 1043, 1044, 1045], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1191, 1192, 1193, 1194], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1216, 1217, 1218, 1219], [2210, 2211, 2212, 2213]
[1271, 1272, 1273, 1274], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1321], [3722, 5222]
[13

In [37]:
# calculate a dataframe with the cos sim value for the test tunes
df_tfidf = pd.DataFrame(columns=['tune1', 'tune2', 'ngram_size', 'section1', 'section2', 'similarity'])
for ngram_size, tfidf_m in tfidf_model.items():

    for tune1, tune2 in contrafacts_test:
      #print(f"tune1: {tune1}, tune2: {tune2}")
      tune_sim = []
      # loop over all combinations of sections
      for s1 in tune1:
          for s2 in tune2:
                #print(f"ngram: {ngram_size}: {sectionid_to_section[s1]}, {sectionid_to_section[s2]}:  {cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]}")
                row = [sectionid_to_title_id[tune1[0]], sectionid_to_title_id[tune2[0]], ngram_size, sectionid_to_section[s1], sectionid_to_section[s2], cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]]
                df_tfidf.loc[len(df_tfidf)] = row
      


In [38]:
df_tfidf.query('ngram_size == 1').sort_values(['tune1', 'similarity'], ascending=[True, False]).head(50)

2021-11-18 20:01:19,109 : INFO : NumExpr defaulting to 2 threads.


,tune1,tune2,ngram_size,section1,section2,similarity
10,0,251,1,"26-2, section3 (B)","Confirmation, section3 (B)",0.691861
6,0,251,1,"26-2, section2 (A)","Confirmation, section3 (B)",0.502416
14,0,251,1,"26-2, section4 (A)","Confirmation, section3 (B)",0.502416
2,0,251,1,"26-2, section1 (A)","Confirmation, section3 (B)",0.498167
3,0,251,1,"26-2, section1 (A)","Confirmation, section4 (A)",0.417841
1,0,251,1,"26-2, section1 (A)","Confirmation, section2 (A)",0.398857
0,0,251,1,"26-2, section1 (A)","Confirmation, section1 (A)",0.382584
11,0,251,1,"26-2, section3 (B)","Confirmation, section4 (A)",0.277554
9,0,251,1,"26-2, section3 (B)","Confirmation, section2 (A)",0.270191
8,0,251,1,"26-2, section3 (B)","Confirmation, section1 (A)",0.248027


In [39]:
df_tfidf_pivot = pd.pivot_table(df_tfidf, index = ['tune1', 'tune2', 'section1', 'section2'], columns = 'ngram_size', values = 'similarity')
df_tfidf_pivot.head(50)

ngram_size                                                                               1  ...         4
tune1 tune2 section1                        section2                                        ...          
0     251   26-2, section1 (A)              Confirmation, section1 (A)            0.382584  ...  0.372820
                                            Confirmation, section2 (A)            0.398857  ...  0.379749
                                            Confirmation, section3 (B)            0.498167  ...  0.390971
                                            Confirmation, section4 (A)            0.417841  ...  0.387522
            26-2, section2 (A)              Confirmation, section1 (A)            0.166310  ...  0.142539
                                            Confirmation, section2 (A)            0.216111  ...  0.155761
                                            Confirmation, section3 (B)            0.502416  ...  0.500740
                                            Confirmation, section4 (A)            0.231294  ...  0.166976
            26-2, section3 (B)              Confirmation, section1 (A)            0.248027  ...  0.185885
                                            Confirmation, section2 (A)            0.270191  ...  0.203248
                                            Confirmation, section3 (B)            0.691861  ...  0.726261
                                            Confirmation, section4 (A)            0.277554  ...  0.203180
            26-2, section4 (A)              Confirmation, section1 (A)            0.166310  ...  0.142539
                                            Confirmation, section2 (A)            0.216111  ...  0.155761
                                            Confirmation, section3 (B)            0.502416  ...  0.500740
                                            Confirmation, section4 (A)            0.231294  ...  0.166976
3     505   52nd Street Theme, section1 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
            52nd Street Theme, section2 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
            52nd Street Theme, section3 (B) I Got Rhythm, section1 (A)            0.240516  ...  0.127823
                                            I Got Rhythm, section2 (A)            0.223873  ...  0.129214
                                            I Got Rhythm, section3 (B)            0.294358  ...  0.318568
                                            I Got Rhythm, section4 (A)            0.186662  ...  0.101162
            52nd Street Theme, section4 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
30    63    Ablution, section1 (A)          All The Things You Are, section1 (A)  0.843729  ...  0.527422
                                            All The Things You Are, section2 (B)  0.052460  ...  0.076243
                                            All The Things You Are, section3 (C)  0.000000  ...  0.000000
                                            All The Things You Are, section4 (

In [40]:
df_tfidf_pivot.query('tune1 == 1170')

ngram_size                                                                     1  ...         4
tune1 tune2 section1                           section2                           ...          
1170  159   These Foolish Things, section1 (A) Blue Moon, section1 (A)  0.782701  ...  0.680510
                                               Blue Moon, section2 (A)  0.670174  ...  0.615911
                                               Blue Moon, section3 (B)  0.419513  ...  0.297998
                                               Blue Moon, section4 (A)  0.782701  ...  0.680510
            These Foolish Things, section2 (A) Blue Moon, section1 (A)  0.675141  ...  0.648572
                                               Blue Moon, section2 (A)  0.586182  ...  0.593161
                                               Blue Moon, section3 (B)  0.426671  ...  0.370118
                                               Blue Moon, section4 (A)  0.675141  ...  0.648572
            These Foolish Things, section3 (B) Blue Moon, section1 (A)  0.232981  ...  0.226942
                                               Blue Moon, section2 (A)  0.200085  ...  0.201910
                                               Blue Moon, section3 (B)  0.189414  ...  0.151101
                                               Blue Moon, section4 (A)  0.232981  ...  0.226942
            These Foolish Things, section4 (A) Blue Moon, section1 (A)  0.774064  ...  0.680519
                                               Blue Moon, section2 (A)  0.659705  ...  0.620852
                                               Blue Moon, section3 (B)  0.516911  ...  0.391904
                                               Blue Moon, section4 (A)  0.774064  ...  0.680519

[16 rows x 4 columns]

In [41]:
df_tfidf_pivot.query('tune1 == 1138')

ngram_size                                                                            1  ...         4
tune1 tune2 section1                          section2                                   ...          
1138  206   Sweet Sue, Just You, section1 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122
            Sweet Sue, Just You, section2 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122
            Sweet Sue, Just You, section3 (B) Bye Bye Blackbird, section1 (A)  0.165758  ...  0.229472
                                              Bye Bye Blackbird, section2 (B)  0.287453  ...  0.376501
                                              Bye Bye Blackbird, section3 (C)  0.317302  ...  0.642809
            Sweet Sue, Just You, section4 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122

[12 rows x 4 columns]

# SVD / LSI Example

In [42]:
data_ngrams = list_to_ngrams(data, n=4)

for line in data_ngrams[:10]:
  print(line)

['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-Eb7', 'C7-FM7-Eb7-G#M7', 'FM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-CM7']
['Gm7-C7-Bm7-E7', 'C7-Bm7-E7-AM7', 'Bm7-E7-AM7-C7', 'E7-AM7-C7-FM7', 'AM7-C7-FM7-Bbm7', 'C7-FM7-Bbm7-Eb7', 'FM7-Bbm7-Eb7-G#M7', 'Bbm7-Eb7-G#M7-Dm7', 'Eb7-G#M7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-Eb7', 'C7-FM7-Eb7-G#M7', 'FM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-CM7']
['Am7-Am7-Cm7-Cm7', 'Am7-Cm7-Cm7-EbM7', 'Cm7-Cm7-EbM7-EbM7', 'Cm7-EbM7-EbM7-Em7b5', 'EbM7-EbM7-Em7b5-A7', 'EbM7-Em7b5-A7-Dm7', 'Em7b5-A7-Dm7-Dm7', 'A7-Dm7-Dm7-Bm

In [43]:
from gensim import corpora, models, similarities
from collections import defaultdict

frequency = defaultdict(int)
for text in data_ngrams:
    for token in text:
        frequency[token] += 1

data_ngrams = [[token for token in text if frequency[token] > 1] for text in data_ngrams]
dictionary = corpora.Dictionary(data_ngrams)

# doc2bow counts the number of occurences of each distinct word,
# converts the word to its integer word id and returns the result
# as a sparse vector

corpus = [dictionary.doc2bow(text) for text in data_ngrams]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)  # num_topics can be maximum the size of the number of unique tokens

2021-11-18 20:01:19,401 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-18 20:01:19,527 : INFO : built Dictionary(8838 unique tokens: ['Am7-D7-Dm7-G7', 'B7-EM7-G7-Gm7', 'CM7-Eb7-G#M7-B7', 'E7-Am7-D7-Dm7', 'EM7-G7-Gm7-C7']...) from 5403 documents (total 54912 corpus positions)
2021-11-18 20:01:19,529 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(8838 unique tokens: ['Am7-D7-Dm7-G7', 'B7-EM7-G7-Gm7', 'CM7-Eb7-G#M7-B7', 'E7-Am7-D7-Dm7', 'EM7-G7-Gm7-C7']...) from 5403 documents (total 54912 corpus positions)", 'datetime': '2021-11-18T20:01:19.529204', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:01:19,708 : INFO : using serial LSI version on this node
2021-11-18 20:01:19,710 : INFO : updating model with new documents
2021-11-18 20:01:19,711 : INFO : preparing a new chunk of documents
2021-11-18 20:01:19,755 : INFO : 

In [44]:
lsi.get_topics().shape

(200, 8838)

In [45]:
title_to_sectionid['These Foolish Things']

[3866, 3867, 3868, 3869]

In [46]:
# add new sample

i = 3868
print(f"Check similarity for {sectionid_to_section[i]}")
tune = data_ngrams[i]
vec_bow = dictionary.doc2bow(tune)

# convert the query to LSI space
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
print(len(vec_lsi))

2021-11-18 20:01:21,646 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


Check similarity for These Foolish Things, section3 (B)


2021-11-18 20:01:22,097 : INFO : creating matrix with 5403 documents and 200 features


200


In [47]:
# perform a similarity query against the corpus
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

print(sims)
print(len(sims))

[(3868, 1.0), (2032, 0.9965869), (825, 0.91630465), (3595, 0.90807617), (1941, 0.8724793), (1986, 0.7942096), (1987, 0.7942096), (1989, 0.7942096), (3881, 0.7895956), (3985, 0.77766764), (1867, 0.77565324), (4306, 0.77054745), (1906, 0.7705351), (888, 0.76466775), (781, 0.75871366), (2361, 0.71789634), (1473, 0.7149668), (1672, 0.71338844), (1787, 0.69060135), (2719, 0.6810168), (252, 0.6745573), (1606, 0.67413014), (1724, 0.66863674), (2324, 0.66794276), (2350, 0.6478162), (1368, 0.6339495), (4267, 0.63035923), (3245, 0.61947787), (1877, 0.6005107), (49, 0.5983696), (3032, 0.5881171), (991, 0.5603283), (1153, 0.55082345), (1716, 0.54989874), (1570, 0.54945743), (1980, 0.5479647), (1717, 0.5332315), (3903, 0.529471), (3242, 0.52854025), (102, 0.5251451), (1878, 0.5249256), (105, 0.52173483), (1729, 0.5081425), (3875, 0.5058158), (3757, 0.50558835), (205, 0.505476), (38, 0.50516117), (2447, 0.5045794), (3628, 0.4997662), (3630, 0.4997662), (3600, 0.4973355), (3221, 0.48173028), (1110, 0

In [48]:
print(f"Similarity for '{sectionid_to_section[i]}'")
print()

for id, sim in sims[:15]:
  print(f"{sim:.3f} {sectionid_to_section[id]}")

Similarity for 'These Foolish Things, section3 (B)'

1.000 These Foolish Things, section3 (B)
0.997 Isn't It A Pity, section3 (B)
0.916 Coral, section0 (nan)
0.908 Soultrane, section3 (B)
0.872 Imagination, section3 (B)
0.794 In Your Own Sweet Way, section1 (A)
0.794 In Your Own Sweet Way, section2 (A)
0.794 In Your Own Sweet Way, section4 (A)
0.790 They Say It's Wonderful, section3 (B)
0.778 Too Young To Go Steady, section3 (B)
0.776 I've Got My Love To Keep Me Warm, section3 (B)
0.771 Yardbird Suite, section3 (B)
0.771 If I Had You, section3 (B)
0.765 Cynthia's In Love, section3 (B)
0.759 Come Back To Me, section3 (B)


#### Test for single tunes

In [49]:
# test a sample

n = 4

for tune in tunes_eval_list:
  print(f"\n\n*** {tune} ***")
  for s1 in title_to_sectionid[tune]:

    print(f"\nCheck similarity for {sectionid_to_section[s1]}")
    tune = data_ngrams[s1]
    vec_bow = dictionary.doc2bow(tune)

    # convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    index = similarities.MatrixSimilarity(lsi[corpus])

    # perform a similarity query against the corpus
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])

    for id, sim in sims[:15]:
      print(f"{sim:.3f} {sectionid_to_section[id]}")

2021-11-18 20:01:23,159 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)




*** Sweet Sue, Just You ***

Check similarity for Sweet Sue, Just You, section1 (A)


2021-11-18 20:01:23,694 : INFO : creating matrix with 5403 documents and 200 features
2021-11-18 20:01:24,749 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Sweet Sue, Just You, section1 (A)
1.000 Sweet Sue, Just You, section2 (A)
1.000 Sweet Sue, Just You, section4 (A)
0.655 A Blossom Fell, section1 (A)
0.651 I Mean You, section5 (A)
0.603 How Are Things In Glocca Morra, section2 (A)
0.589 My Secret Love, section2 (A)
0.573 Someday (You'll Be Sorry), section2 (B)
0.562 Seven Come Eleven, section1 (A)
0.562 Seven Come Eleven, section2 (A)
0.562 Seven Come Eleven, section4 (A)
0.559 Breeze And I, The, section1 (A)
0.559 Breeze And I, The, section2 (A)
0.558 Airmail Special, section1 (intro)
0.558 I Want To Be Happy, section2 (A)

Check similarity for Sweet Sue, Just You, section2 (A)


2021-11-18 20:01:25,321 : INFO : creating matrix with 5403 documents and 200 features
2021-11-18 20:01:26,420 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


...

1.000 Misty, section2 (A)
0.789 Misty, section4 (A)
0.760 My Way, section1 (A)
0.760 My Way, section2 (A)
0.739 I Remember You, section2 (A)
0.738 Easy Street, section3 (B)
0.727 Portrait Of Jennie, section2 (A)
0.695 My Secret Love, section3 (B)
0.693 You're Everything, section1 (A)
0.684 Mahjong, section5 (C)
0.675 Breeze And I, The, section4 (C)
0.668 Taking A Chance On Love, section3 (B)
0.653 It's Easy To Remember, section3 (B)
0.649 You're Everything, section3 (C)
0.642 Do Nothin' Til You Hear From Me, section2 (A)

Check similarity for Misty, section3 (B)


2021-11-18 20:02:40,323 : INFO : creating matrix with 5403 documents and 200 features
2021-11-18 20:02:41,334 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Misty, section3 (B)
0.727 Upper Manhattan Medical Group, section4 (A)
0.727 Upper Manhattan Medical Group, section2 (A)
0.723 Upper Manhattan Medical Group, section1 (A)
0.720 This Can't Be Love, section3 (B)
0.707 A Sound For Sore Ears, section3 (C)
0.706 Laura, section1 (A)
0.706 Laura, section3 (A)
0.705 S.O.S., section5 (D)
0.702 P.S. I Love You, section3 (B)
0.693 Strike Up The Band, section2 (B)
0.683 Só Danço Samba, section3 (B)
0.662 In A Mellow Tone (In A Mellotone), section1 (A)
0.662 In A Mellow Tone (In A Mellotone), section3 (A)
0.652 From This Moment On, section1 (A)

Check similarity for Misty, section4 (A)


2021-11-18 20:02:41,879 : INFO : creating matrix with 5403 documents and 200 features


1.000 Misty, section4 (A)
0.789 Misty, section2 (A)
0.710 Portrait Of Jennie, section4 (A)
0.698 Do Nothin' Til You Hear From Me, section4 (A)
0.694 My Way, section1 (A)
0.694 My Way, section2 (A)
0.691 Easy Street, section3 (B)
0.688 I Remember You, section2 (A)
0.677 Portrait Of Jennie, section2 (A)
0.650 My Secret Love, section3 (B)
0.641 You're Everything, section1 (A)
0.638 I Remember You, section4 (A)
0.637 Mahjong, section5 (C)
0.626 Taking A Chance On Love, section3 (B)
0.624 More I See You, The, section2 (B)


# Doc2Vec Embeddings

## Read and Tag Data

In [50]:
df.head()

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 G#7 C#M7 E7 ...
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm7 C7 Bm7 E7 AM7 C7 FM7 Bbm7 Eb7 G#M7 Dm7 G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am7 Am7 Cm7 Cm7 EbM7 EbM7 Em7b5 A7 Dm7 Dm7 Bm7...


In [63]:

for i, row in df.iterrows():
  tokens = []
  for ngram in range(3):
    tokens.extend(ngrams(row['chords'].split(), ngram, '-'))
  


[[], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7'], [], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-CM7'], [], ['Gm7', 'C7', 'Bm7', 'E7', 'AM7', 'C7', 'FM7', 'Bbm7', 'Eb7', 'G#M7', 'Dm7', 'G7'], ['Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-AM7', 'AM7-C7', 'C7-FM7', 'FM7-Bbm7', 'Bbm7-Eb7', 'Eb7-G#M7', 'G#M7-Dm7', 'Dm7-G7'], [], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-CM7'], [

In [97]:
def read_corpus(df, max_ngram_size, tokens_only=False):
    # concatenate unigrams, bigrams and trigrams and use them as input; use PV-DBOW since it does not depend on the order
    for i, row in df.iterrows():
      tokens = []
      for ngram in range(max_ngram_size + 1):
        tokens.extend(ngrams(row['chords'].split(), ngram, '-'))
      if tokens_only:
          yield tokens
      else:
          # for training data, add tags
          yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id'], row['tune_id']])


## Doc2Vec Hyperparameters

In [79]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [80]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [81]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': [#'pv-dm', 'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 #2, 
                                 3, 
                                 4, 
                                 10
                                 ],
                      'ngrams': [
                                 #1, 
                                 #2, 
                                 3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=40, 
    min_count=1,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(df, max_ngram_size=3))

            # evaluate different window sizes
            for window in params['window']:

                # calculate the model
                if algo == 'd2v':
                    print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}")
                    print(f"Corpus: {train_corpus[0]}")
                    model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                          dm = dm,
                                                          dm_concat = concat,
                                                          )
                    model.build_vocab(train_corpus)
                    model.train(train_corpus, 
                                total_examples=model.corpus_count, 
                                epochs=model.epochs)

                else:
                    model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                model.save(file_name + '.model')

                file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                with open(file_name + '.train', 'wb') as f:
                      # Pickle the 'data' dictionary using the highest protocol available.
                      pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-18 20:20:15,148 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n10,t2)', 'datetime': '2021-11-18T20:20:15.148491', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:20:15,150 : INFO : collecting all words and their counts
2021-11-18 20:20:15,156 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-18 20:20:15,224 : INFO : collected 15908 word types and 5403 unique tags from a corpus of 5403 examples and 232161 words
2021-11-18 20:20:15,226 : INFO : Creating a fresh vocabulary
2021-11-18 20:20:15,304 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 15908 unique words (100.0%% of original 15908, drops 0)', 'datetime': '2021-11-18T20:20:15.304718', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=3
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:20:15,428 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:20:15,432 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:20:15,435 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:20:15.435115', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:20:15,651 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:20:15,654 : INFO : resetting layer weights
2021-11-18 20:20:15,673 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:20:15.673447', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=4
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:21:05,811 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:21:05,813 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:21:05,815 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:21:05.815335', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:21:06,025 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:21:06,027 : INFO : resetting layer weights
2021-11-18 20:21:06,048 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:21:06.048543', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=10
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:21:52,764 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:21:52,767 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:21:52,768 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:21:52.768863', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:21:52,971 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:21:52,973 : INFO : resetting layer weights
2021-11-18 20:21:52,995 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:21:52.995036', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

In [82]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}


for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    if not "npy" in name: ## !!! concat_x_y.model.syn1neg.npy cannot be loaded
      models[name] = Doc2Vec.load(file_name)

2021-11-18 20:23:07,478 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model
2021-11-18 20:23:07,514 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model.dv.* with mmap=None
2021-11-18 20:23:07,516 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model.wv.* with mmap=None
2021-11-18 20:23:07,519 : INFO : setting ignored attribute cum_table to None


['autos_d2v_pv-dbow_3_10.model', 'autos_d2v_pv-dbow_3_4.model', 'autos_d2v_pv-dbow_3_3.model']
./models_doc2vec/autos_d2v_pv-dbow_3_10.model


2021-11-18 20:23:07,707 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_10.model', 'datetime': '2021-11-18T20:23:07.706994', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:23:07,710 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model
2021-11-18 20:23:07,762 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.dv.* with mmap=None
2021-11-18 20:23:07,763 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.wv.* with mmap=None
2021-11-18 20:23:07,765 : INFO : setting ignored attribute cum_table to None


./models_doc2vec/autos_d2v_pv-dbow_3_4.model


2021-11-18 20:23:07,991 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4.model', 'datetime': '2021-11-18T20:23:07.991182', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:23:07,992 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model
2021-11-18 20:23:08,060 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model.dv.* with mmap=None
2021-11-18 20:23:08,063 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model.wv.* with mmap=None
2021-11-18 20:23:08,065 : INFO : setting ignored attribute cum_table to None


./models_doc2vec/autos_d2v_pv-dbow_3_3.model


2021-11-18 20:23:08,263 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_3.model', 'datetime': '2021-11-18T20:23:08.263048', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [83]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dbow_3_3.train', 'autos_d2v_pv-dbow_3_10.train', 'autos_d2v_pv-dbow_3_4.train']
./traindata_doc2vec/autos_d2v_pv-dbow_3_3.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_10.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_4.train


#### Determine Similarity of Tunes

In [84]:
#assert(len(model.dv.vectors == len(list(title_to_sectionid.keys()))))

In [85]:
tunes_eval_list = [
  'These Foolish Things', 
]

In [86]:
title_to_sectionid["These Foolish Things"]

[3866, 3867, 3868, 3869]

In [87]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      for model_name, model in models.items():
          print(f"{model_name}")
          sim = model.dv.most_similar(section)
          #print(f"'{sectionid_to_section[section]}' is most similar to:")
          for index, value in sim:
              print(f'\t{value:.3f}: {sectionid_to_section[index]}')

              # don't store results to the own tune
              #if index not in title_to_sectionid[tune]:
              if True:
                  df_sim.loc[len(df_sim)] = [tune, 
                                            model_name, 
                                            sectionid_to_section[section], 
                                            sectionid_to_section[index], 
                                            f"{value:.3f}",
                                             sectionid_to_title[index]]



These Foolish Things

*** These Foolish Things, section1 (A) ***
autos_d2v_pv-dbow_3_10.model
	0.915: These Foolish Things, section4 (A)
	0.889: These Foolish Things, section2 (A)
	0.865: Put On A Happy Face, section2 (B)
	0.798: Nearness Of You, The, section3 (B)
	0.778: Stairway To The Stars, section1 (A)
	0.773: It's A Lovely Day Today, section2 (B)
	0.767: On The Sunny Side Of The Street, section3 (B)
	0.762: Way You Look Tonight, The, section1 (A)
	0.761: Street Of Dreams, section2 (A)
	0.757: Emily, section2 (B)
autos_d2v_pv-dbow_3_4.model
	0.907: These Foolish Things, section4 (A)
	0.890: These Foolish Things, section2 (A)
	0.852: Put On A Happy Face, section2 (B)
	0.793: Nearness Of You, The, section3 (B)
	0.777: It's A Lovely Day Today, section2 (B)
	0.774: Stairway To The Stars, section1 (A)
	0.767: Way You Look Tonight, The, section1 (A)
	0.764: Street Of Dreams, section2 (A)
	0.758: Emily, section2 (B)
	0.754: On The Sunny Side Of The Street, section3 (B)
autos_d2v_pv-dbow_

In [88]:
from google.colab import files

file = 'doc2vec_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [89]:
df_sim.sort_values(['sim_value'], ascending=False).head(50)

,title1,model,ref_section,sim_section,sim_value,sim_title
90,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.938,These Foolish Things
30,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.938,These Foolish Things
110,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.936,These Foolish Things
50,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.936,These Foolish Things
40,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.931,These Foolish Things
100,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.931,These Foolish Things
111,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section4 (A)","These Foolish Things, section1 (A)",0.921,These Foolish Things
20,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section1 (A)","These Foolish Things, section4 (A)",0.921,These Foolish Things
0,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section1 (A)","These Foolish Things, section4 (A)",0.915,These Foolish Things
91,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section4 (A)","These Foolish Things, section1 (A)",0.915,These Foolish Things


In [90]:
df_sim.loc[:, 'model'].unique()

array(['autos_d2v_pv-dbow_3_10.model', 'autos_d2v_pv-dbow_3_4.model',
       'autos_d2v_pv-dbow_3_3.model'], dtype=object)

In [91]:
dd = df_sim.query(f'model == "autos_d2v_pv-dm_4_4.model"').sort_values('sim_value', ascending=False).head(50)

In [92]:
dd.value_counts('sim_title')

Series([], dtype: int64)

In [93]:
for model_name in sorted(models):
  if "dbow_" in model_name:
    print(f"\n*** {model_name} ***")
    dd = df_sim.query(f'model == "{model_name}"').sort_values('sim_value', ascending=False).head(50)
    dd_count = dd.value_counts('sim_title')
    print(dd_count)


*** autos_d2v_pv-dbow_3_10.model ***
sim_title
These Foolish Things                 6
Emily                                4
Put On A Happy Face                  3
It's A Lovely Day Today              3
Way You Look Tonight, The            3
Rosetta                              2
Mountain Greenery                    2
Let's Fall In Love                   1
I Loves You Porgy                    1
I'm Getting Sentimental Over You     1
I've Told Eve'ry Little Star         1
If I Had You                         1
Imagination                          1
Isn't It A Pity                      1
Why Do I Love You                    1
More Than You Know                   1
Old Cape Cod                         1
On The Sunny Side Of The Street      1
Stairway To The Stars                1
Street Of Dreams                     1
They Can't Take That Away From Me    1
Too Young To Go Steady               1
Walkin' My Baby Back Home            1
Nearness Of You, The                 1
dtype: int64

**

Try out a model on multiple tunes

In [94]:
best_model = 'autos_d2v_pv-dbow_3_4.model'
model = models[best_model]

In [95]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [96]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      sim = model.dv.most_similar(section)
      #print(f"'{sectionid_to_section[section]}' is most similar to:")
      for index, value in sim:
          print(f'\t{value:.3f}: {sectionid_to_section[index]}')

          # don't store results to the own tune
          #if index not in title_to_sectionid[tune]:
          if True:
              df_sim.loc[len(df_sim)] = [tune, 
                                        model_name, 
                                        sectionid_to_section[section], 
                                        sectionid_to_section[index], 
                                        f"{value:.3f}",
                                          sectionid_to_title[index]]



Sweet Sue, Just You

*** Sweet Sue, Just You, section1 (A) ***
	0.998: Sweet Sue, Just You, section2 (A)
	0.998: Sweet Sue, Just You, section4 (A)
	0.908: Moten Swing, section4 (A)
	0.904: Honeysuckle Rose, section2 (A)
	0.901: Moten Swing, section1 (A)
	0.897: Seven Come Eleven, section4 (A)
	0.891: Seven Come Eleven, section1 (A)
	0.890: Seven Come Eleven, section2 (A)
	0.878: We See, section3 (B)
	0.871: Perdido, section2 (A)

*** Sweet Sue, Just You, section2 (A) ***
	0.998: Sweet Sue, Just You, section1 (A)
	0.998: Sweet Sue, Just You, section4 (A)
	0.909: Moten Swing, section4 (A)
	0.908: Honeysuckle Rose, section2 (A)
	0.903: Moten Swing, section1 (A)
	0.897: Seven Come Eleven, section4 (A)
	0.890: Seven Come Eleven, section1 (A)
	0.890: Seven Come Eleven, section2 (A)
	0.877: We See, section3 (B)
	0.871: Perdido, section2 (A)

*** Sweet Sue, Just You, section3 (B) ***
	0.847: Sweet Sue, section3 (B)
	0.814: Easy To Love, section2 (B)
	0.735: Save Your Love For Me, section2 (A)


# Good Matches

* Misty - Portrait Of Jennie
* Misty - September in the Rain
* Misty - I May Be Wrong


